<a href="https://colab.research.google.com/github/SergheiMihailov/ml-project-cassava/blob/main/ml_project_v2_putting_it_all_together.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Installs
!pip install -q -U keras-tuner

In [21]:
# Imports
import gdown
import os
import json
import csv   
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import scipy.misc
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import tensorflow.keras.layers.experimental.preprocessing as keras_preproc
import kerastuner as kt

In [22]:
# Constants
# Data
# full = original + augmented
TRAIN_IMAGES_PATH = 'full_train_images/'
TRAIN_LABELS_PATH = 'full_train.csv'
LABEL_TO_DISEASE_MAP_PATH = 'label_num_to_disease_map.json'
CONFUSION_MATRIX_PATH = 'images'
MODEL_CP_PATH = 'models'

N_CLASSES = 5

# Images
IMG_SIZE = 224
SIZE = (IMG_SIZE,IMG_SIZE)
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Training
EPOCHS_TO_TRAIN = 40
BATCH_SIZE = 16
N_CV_SPLITS = 3


In [23]:
# Downloading data
if not os.path.exists(TRAIN_LABELS_PATH) or not os.path.exists(LABEL_TO_DISEASE_MAP_PATH):
  !gdown --id "1xbEVK_NigW_5ngwKMHvuOTehYhT2v2WF" # original train.csv
  !wget files.brainfriz.com/aug_train.csv         # augmented train.csv
  !wget files.brainfriz.com/full_train.csv        # full train -> augmented + original
  !gdown --id "1SvI9dN2_25c2OlevwK4TjmzBNysjE_PO" # label mapping

if not os.path.exists(TRAIN_IMAGES_PATH):
  !wget files.brainfriz.com/train_images.zip      # original images
  !wget files.brainfriz.com/augmented_images.zip  # augmented images
  os.chdir('/content/')
  !unzip -j -qq -o train_images.zip -d full_train_images
  !unzip -j -qq -o augmented_images.zip -d full_train_images
  
if not os.path.exists(CONFUSION_MATRIX_PATH):
  !mkdir images #for the confusion matrix images

if not os.path.exists(MODEL_CP_PATH):
  !mkdir models # for saved models

In [24]:
# Dataset
labels = pd.read_csv(TRAIN_LABELS_PATH)
f = open(LABEL_TO_DISEASE_MAP_PATH)
label_class_name = json.load(f)
label_class_name = {int(k):v for k,v in label_class_name.items()}
labels['class_name'] = labels.label.map(label_class_name)

def getImagesByIds(labels_with_image_id):
  return ImageDataGenerator().flow_from_dataframe(
      labels_with_image_id,
      subset='training',
      directory = TRAIN_IMAGES_PATH,
      x_col = 'image_id',
      y_col = 'class_name',
      target_size = SIZE,
      color_mode='rgb',
      class_mode = 'categorical',
      batch_size = BATCH_SIZE)

def getDatasetsForCrossValidationSplit(n_splits, split_index, shuffle=False, split_size=None):
  kfold = StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state=42)

  train_indices, val_indices = list(kfold.split(labels['image_id'], labels['label']))[split_index]
  
  if split_size:
    train_indices = train_indices[:split_size*n_splits]
    val_indices = val_indices[:split_size]

  train_ds = labels.iloc[train_indices]
  val_ds = labels.iloc[val_indices]

  train_set = getImagesByIds(train_ds)
  val_set = getImagesByIds(val_ds)

  return train_set, val_set

In [59]:
# Model definition
def getEfficientNetB0():
  return [
      # architecture
      tf.keras.applications.EfficientNetB0(
        include_top=True, weights=None, input_tensor=None,
        input_shape=INPUT_SHAPE, pooling=None, classes=N_CLASSES,
        classifier_activation='softmax', drop_connect_rate=0.4
      ),
      # preprocess_input
      tf.keras.applications.efficientnet.preprocess_input
  ]

def getResNet50V2(): 
  return [
      # architecture 
      tf.keras.applications.ResNet50V2(
        include_top=True, weights=None, input_tensor=None,
        input_shape=INPUT_SHAPE, pooling=None, classes=N_CLASSES,
        classifier_activation='softmax'
      ),
      # preprocess_input
      tf.keras.applications.resnet_v2.preprocess_input
  ]

def getMobileNetV3Small(): 
  return [
      # architecture
      tf.keras.applications.MobileNetV3Small(
        input_shape=INPUT_SHAPE, alpha=1, minimalistic=True, include_top=True,
        weights=None, input_tensor=None, classes=5, pooling='avg',
        dropout_rate=0, classifier_activation='softmax'
      ),
      # preprocess_input:
      tf.keras.applications.mobilenet_v3.preprocess_input
  ]

def model_builder(hp):
  architecture, preprocess_input = getEfficientNetB0()

  input_layer = preprocess_input(tf.keras.layers.Input(shape=INPUT_SHAPE))

  model = tf.keras.Model(input_layer, architecture(input_layer))
  
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])
  hp_label_smoothing = hp.Choice('label_smoothing', values=[1e-3, 1e-4, 1e-5, 0.0])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.CategoricalCrossentropy(label_smoothing=hp_label_smoothing),
                metrics=['accuracy'])

  return model

# callbacks = tf.keras.callbacks.CallbackList(
#     callbacks=None, add_history=True, add_progbar=True
# )

# callbacks.append(cb for cb in [
#   tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
#   tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                                 patience=5, min_lr=0.001),
#   tf.keras.callbacks.ModelCheckpoint(
#       MODEL_CP_PATH, monitor='val_loss', verbose=0, save_best_only=False,
#       save_weights_only=False, mode='auto', save_freq='epoch',
#       options=None
#   )
# ])

tuning_callbacks = [
  tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
  tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=5, min_lr=0.00001),
  # tf.keras.callbacks.ModelCheckpoint(
  #     MODEL_CP_PATH, monitor='val_loss', verbose=0, save_best_only=False,
  #     save_weights_only=False, mode='auto', save_freq='epoch',
  #     options=None
  # )
]

In [42]:
!rm -rf hyperparams

In [61]:
# Hyperparameter tuning
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='hyperparams_2',
                     project_name='cassava'
                     )

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

train_set, val_set = getDatasetsForCrossValidationSplit(n_splits=N_CV_SPLITS ,split_index=0, shuffle=True, split_size=50)
# Tune hyperparameters on first cross-validation (refactor later to use saved hps)
tuner.search(train_set, epochs=EPOCHS_TO_TRAIN, validation_data=val_set,callbacks=tuning_callbacks)

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters()[0]
print(f"""
The hyperparameter search is complete. Optimal values: \n
learning_rate: {best_hps.get('learning_rate')}; \n
label_smoothing: {best_hps.get('label_smoothing')}; \n
""")

Trial 22 Complete [00h 00m 31s]
val_accuracy: 0.5600000023841858

Best val_accuracy So Far: 0.5600000023841858
Total elapsed time: 00h 09m 42s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit



The hyperparameter search is complete. Optimal values: 

learning_rate: 0.001; 

label_smoothing: 1e-05; 




In [55]:
# !zip -r hyperparams.zip hyperparams

In [62]:
# Evaluation definitions 
class ConfusionMatrix(keras.callbacks.Callback):
  def __init__(self, val_set, val_y):
    self.val_set = val_set
    self.val_y = val_y
    self.counter = 0

  def on_epoch_end(self, epoch, logs=None):
    self.plot()
    self.counter += 1

  def plot(self):
    test_pred_raw = self.model.predict(self.val_set)
    test_pred = np.argmax(test_pred_raw, axis=1)

    cm = sklearn.metrics.confusion_matrix(self.val_y, test_pred)
    self.plot_confusion_matrix(cm, class_names=[0,1,2,3,4])

  def plot_confusion_matrix(self, cm, class_names):
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      color = "white" if cm[i, j] > threshold else "black"
      plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig('images/conf{0}.png'.format(self.counter)) #save file
    plt.close()

# plotter = ConfusionMatrix(val_set, val_set.classes);

In [ ]:
# Training & evaluating the model
best_hps = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_hps)

training_callbacks = [
  tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
  tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=5, min_lr=0.00001),
  tf.keras.callbacks.ModelCheckpoint(
      MODEL_CP_PATH, monitor='val_loss', verbose=0, save_best_only=True,
      save_weights_only=False, mode='auto', save_freq='epoch',
      options=None
  )
]

for i in range(N_CV_SPLITS):
  train_set, val_set = getDatasetsForCrossValidationSplit(n_splits=N_CV_SPLITS, split_index=i)

  plotter = ConfusionMatrix(val_set, val_set.classes);

  history = model.fit(
        train_set,
        steps_per_epoch=train_set.n // BATCH_SIZE,
        validation_data=val_set,
        epochs=EPOCHS_TO_TRAIN,
        callbacks = [plotter, *training_callbacks]
  )

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Found 43859 validated image filenames belonging to 5 classes.
Found 21930 validated image filenames belonging to 5 classes.
Epoch 1/40
   6/2741 [..............................] - ETA: 13:17 - loss: 5.2219 - accuracy: 0.2101WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1089s vs `on_train_batch_end` time: 0.1928s). Check your callbacks.


 100/2741 [>.............................] - ETA: 12:30 - loss: 3.5059 - accuracy: 0.2571

In [52]:
# The End

In [50]:
# Our own model